En el siguiente ejemplo vamos a implementar una CNN que permita clasificacion multiclase.

El primer paso es crear 2 carpetas que son: la carpeta de entrenamiento y la carpeta de pruebas; luego dentro de cada una de ellas se deben ubicar N carpetas que denotan cada una de las clases, y dentro de cada una de estas carpetas deben  contener las respectivas imagenes correspondientes a la clase. 

![folder_data](img/folder_data.jpeg)

Keras tiene una clase llamada ImageDataGenerator que permite hacer aumentos en tiempo real de las imagenes.En este ejmplo vamos a utilizar el metodo flow_from_directory().

Es muy importante resaltar que el metodo flow_from_directory() espera una estructura similar a la que se presentan en la figura anterior para su correcto funcionamiento.

Los nombres de las carpetas para las clases son importantes, nombrarlas (o renombrarlas) con los nombres de las etiquetas respectivas para que sea más fácil para usted más tarde.

El primer paso es crear una instancia de la clase ImageDataGenerator con algunas configuraciones basicas, como rotacion de 40 grados de la imagen, normalizacion de la imagen, acercamiento aleatorio de la imagen entre otros opciones disponibles para su consulta en el siguiente enlace:https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [12]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
!pip install pillow

In [13]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [14]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
ENTRENAMIENTO_DIR = 'data/data/entrenamiento/'
TEST_DIR = 'data/data/pruebas/'

A continuacion se presenta la siguiente instruccion con los parametros comunmente utilizados para flow_from_directory:

In [16]:
train_generator = train_datagen.flow_from_directory(
    directory=ENTRENAMIENTO_DIR,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 2002 images belonging to 2 classes.


- directory: ruta donde se encuentran las imagenes.
- target_size: las imagenes de entrada sera redimenzionadas al tamaño que determine.
- color_mode: si la imagen es en escala de grises colocar "grayscale", si es a color "rgb"
- batch_size: numero de imagenes que se procesaran por el generador de lotes.
- class_mode: si es un problema binario configure "binary", si es multi clases configure "categorical"
- shuffle: configure True, si desea barajar las imagenes, de lo contrario configure False.
- seed: Semilla aleatoria para aplicar un aumento de imagen aleatorio y barajar el orden de la imagen.

Igual paso para las imagenes de pruebas:

In [17]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [18]:
validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 400 images belonging to 2 classes.


En el caso anterior se configuro class_mode como None, porque solo requerimos que nos retorne las imagenes. En el caso de shuffle establezca esta opción en False, ya que necesita producir las imágenes en "orden", para predecir las salidas y hacerlas coincidir con sus ids o nombres de archivo únicos.

Creamos la CNN

In [23]:
model = Sequential()
model.add(Convolution2D(32, (3,3), input_shape=(224, 224, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Convolution2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_10 (Activation)   (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
activation_11 (Activation)   (None, 109, 109, 64)      0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 186624)            0         
__________

In [24]:
model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        samples_per_epoch=1000,
        epochs=9,
        validation_data=validation_generator,
        validation_steps=300)

c:\users\jhovanny\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=31, epochs=9, validation_data=<keras_pre..., validation_steps=300)`
  import sys


Epoch 1/9
30/31 [============================>.] - ETA: 3s - loss: 8.2037 - acc: 0.4854

KeyboardInterrupt: 